In [1]:
from bs4 import BeautifulSoup
import psycopg2
import json

https://www.google.com/travel/things-to-do/see-all?dest_mid=%2Fm%2F0cc56&dest_state_type=sattd&dest_src=yts&q=manhattan&ved=0CAAQ8IAIahcKEwig_qz6h-v_AhUAAAAAHQAAAAAQHA&tcfs=EhUKCC9tLzBjYzU2EglNYW5oYXR0YW4

In [2]:
with open('attractions.html', 'r') as file:
    html = file.read()

In [3]:
soup = BeautifulSoup(html, 'html.parser')

In [4]:
divs = soup.find_all('div', class_='skFvHc YmWhbc')

In [5]:
cnt = 0
hots = []
if divs:
    for div in divs:
        div_text = div.get_text(strip=True)
        hots.append(div_text)
        print(cnt, div_text)
        cnt += 1
else:
    print("Div not found")

0 Central Park
1 The Metropolitan Museum of Art
2 Statue of Liberty
3 The Museum of Modern Art
4 Empire State Building
5 Times Square
6 Brooklyn Bridge
7 Solomon R. Guggenheim Museum
8 The Plaza
9 Grand Central Terminal
10 American Museum of Natural History
11 The High Line
12 9/11 Memorial & Museum
13 Rockefeller Center
14 Whitney Museum of American Art
15 Top of The Rock
16 One World Trade Center
17 5th Avenue
18 Bryant Park
19 Intrepid Sea, Air & Space Museum
20 Flatiron Building
21 Washington Square Park
22 St. Patrick's Cathedral
23 Chrysler Building
24 The Morgan Library & Museum
25 Radio City Music Hall
26 New Museum
27 The Frick Collection
28 The Met Cloisters
29 Chelsea Market
30 Madison Square Garden
31 The Jewish Museum
32 Museum of the City of New York
33 Lincoln Center for the Performing Arts
34 New-York Historical Society
35 Wall Street
36 The Battery
37 Tenement Museum
38 New York Public Library - Stephen A. Schwarzman Building
39 Bethesda Terrace
40 Cooper Hewitt, Smith

In [6]:
# connection = psycopg2.connect(
# 	host = your host,
# 	port = "5432",
#     database = 'tourism',
# 	user = username,
# 	password = password
# )
# cursor = connection.cursor()

In [7]:
def create_table(table_name):    
#     connection = psycopg2.connect(
# 	host = your host,
# 	port = "5432",
#     database = 'tourism',
# 	user = username,
# 	password = password
# )
    cursor = connection.cursor()
    create_statement = f'''CREATE TABLE {table_name}(
        id SERIAL PRIMARY KEY,
        housenumber VARCHAR(255),
        street VARCHAR(255),
        postcode VARCHAR(255),
        name VARCHAR(255),
        opening_hours VARCHAR(255),
        phone VARCHAR(255),
        website VARCHAR(255),
        geom GEOMETRY,
        image VARCHAR
    )'''
    
    try:
        cursor.execute(create_statement)
        connection.commit()
        print(f"Table '{table_name}' created successfully.")
    except psycopg2.Error as e:
        print(f"Error creating table '{table_name}': {e}")
    
    cursor.close()
    connection.close()

In [8]:
with open('manhattan_streets.geojson', 'r') as f:
    data = json.load(f)

In [9]:
# create_table('attractions')
# connection.commit()

Change names

In [10]:
hots[22] = "Saint Patrick’s Cathedral"
hots[27] = "Frick Collection"
hots[36] = "Battery Park"
hots[38] = "New York Public Library"
hots[40] = "Cooper–Hewitt, Smithsonian Design Museum"
hots[54] = "Federal Hall National Memorial"
hots[65] = "Cathedral of Saint John the Divine"
hots[68] = "Hudson River Park"
hots[70] = "Roosevelt Island Tramway"
hots[71] = "New York Stock Exchange"
hots[72] = "ICP International Center of Photography Museum"
hots[83] = "Museum of Jewish Heritage"
hots[87] = "Stonewall Inn"

In [13]:
pairs = []
cnt = 0
divs2 = soup.find_all('div', class_='GwjAi')
for div in divs2:
    img_url = div.find_previous_sibling('div', class_='kXlUEb').find('img')['data-src']
    name = hots[cnt]
    pairs.append((img_url, name))
    cnt += 1

cnt = 0
for pair in pairs:
    print(f"{cnt}, Image URL: {pair[0]} Place name: {pair[1]}")
    cnt += 1

0, Image URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQUhfITNdazc_b-fvsQViOkn4alkFX_N7H8dp6TAaDZdWnpSeVr4QO17B1smrXWRJ4QE7Mrixpp9gVxIpMkqQQxIA Place name: Central Park
1, Image URL: https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcS9fsLKfYa6FnZ_BR5jyHCgPRfnoU1BI2T0VQC7HRxkpnj5j8W_QQOBMs1X3MO1HROnBI2vnWil2JuH0mahrqtB_g Place name: The Metropolitan Museum of Art
2, Image URL: https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSUnvDflDt62mAsKzbVvGwJZbdfjd_f3mFwqTxIy713xJbO4obG5_-eDIrgJfgptIe2YKa_thNnJySlUZfDSLK9og Place name: Statue of Liberty
3, Image URL: https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcTKNHGxa9aOKu3eJ632HqNn_9rNipaBe540YKuUcyeC19bfBlkaRsdlPAs75NMsSsoG5WfnFAVqsNGe5B3Uf4P49g Place name: The Museum of Modern Art
4, Image URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGzY3x-IkrTAPI9Pz6MrusBi-FL80-sjrWgU4vFONu4IDnqDcPyyZPJn9L9398UV-JUo5sKrCLcSmDSYsZBPmtWA Place name: Empire State Building
5, Image URL: https://encrypted-tbn3.gstatic.c

In [14]:
cnt = 0
for hot in hots:
    flag = False
    for feature in data['features']:
        name = feature['properties'].get('name', None)
        official_name = feature['properties'].get('official_name', None)
        alt_name = feature['properties'].get('alt_name', None)
        if name is not None and name.lower() == hot.lower():
            housenumber = feature['properties'].get('addr:housenumber', None)
            street = feature['properties'].get('addr:street', None)
            postcode = feature['properties'].get('addr:postcode', None)
            name = feature['properties'].get('name', None)
            opening_hours = feature['properties'].get('opening_hours', None)
            phone = feature['properties'].get('phone', None)
            website = feature['properties'].get('website', None)
            geom = feature['geometry']
            image = pairs[cnt][0]

            insert_cmd = """INSERT INTO attractions
                            (housenumber, street, postcode, name, opening_hours, phone, website, geom, image)
                            VALUES (%s, %s, %s, %s, %s, %s, %s, ST_SetSRID(ST_GeomFromGeoJSON(%s), 4326), %s);"""
            cursor.execute(insert_cmd, (housenumber, street, postcode, name, opening_hours, phone, website, json.dumps(geom), image))
            connection.commit()
            flag = True
            break
        elif official_name is not None and official_name.lower() == hot.lower():
            housenumber = feature['properties'].get('addr:housenumber', None)
            street = feature['properties'].get('addr:street', None)
            postcode = feature['properties'].get('addr:postcode', None)
            name = feature['properties'].get('name', None)
            opening_hours = feature['properties'].get('opening_hours', None)
            phone = feature['properties'].get('phone', None)
            website = feature['properties'].get('website', None)
            geom = feature['geometry']
            image = pairs[cnt][0]

            insert_cmd = """INSERT INTO attractions
                            (housenumber, street, postcode, name, opening_hours, phone, website, geom, image)
                            VALUES (%s, %s, %s, %s, %s, %s, %s, ST_SetSRID(ST_GeomFromGeoJSON(%s), 4326), %s);"""
            cursor.execute(insert_cmd, (housenumber, street, postcode, name, opening_hours, phone, website, json.dumps(geom), image))
            connection.commit()
            flag = True
            break
        elif alt_name is not None and alt_name.lower() == hot.lower():
            housenumber = feature['properties'].get('addr:housenumber', None)
            street = feature['properties'].get('addr:street', None)
            postcode = feature['properties'].get('addr:postcode', None)
            name = feature['properties'].get('name', None)
            opening_hours = feature['properties'].get('opening_hours', None)
            phone = feature['properties'].get('phone', None)
            website = feature['properties'].get('website', None)
            geom = feature['geometry']
            image = pairs[cnt][0]

            insert_cmd = """INSERT INTO attractions
                            (housenumber, street, postcode, name, opening_hours, phone, website, geom, image)
                            VALUES (%s, %s, %s, %s, %s, %s, %s, ST_SetSRID(ST_GeomFromGeoJSON(%s), 4326), %s);"""
            cursor.execute(insert_cmd, (housenumber, street, postcode, name, opening_hours, phone, website, json.dumps(geom), image))
            connection.commit()
            flag = True
            break
    if not flag:
        print(cnt, hot, ": Not found")
    cnt += 1

2 Statue of Liberty : Not found
51 Little Island : Not found


In [12]:
# cursor.execute("ALTER SEQUENCE attractions_id_seq RESTART WITH 1;")
# connection.commit()